In [5]:
import os
import io
import time
import pickle
from subprocess import Popen, PIPE
from functools import reduce
from __future__ import division


In [6]:
given_output=dict()
try:
    given_output = pickle.load(open("given_output.pickle", "rb"))
except (OSError, IOError) as e:

    with open('output.txt') as fp:
        for line in fp:
            line=line.split()
            if(len(line)>=2):
                query_index=line[0]
                doc_ID=line[1]
                if query_index in given_output:
                    given_output[query_index].append(doc_ID)
                else:
                    given_output[query_index]=[doc_ID]
    pickle.dump(given_output, open("given_output.pickle", "wb"))
    fp.close()

In [7]:


path=os.getcwd()
output_file=open("grep_output.txt","w")
time_file=open("Time_grep.txt","w")
precisions=list()
recalls=list()
with open('query.txt') as fp:
    for line in fp:
        query=line.split()
        if(len(query)>=2):
            query_index=query[0]
            query=query[1:]
            doc_list=[]
            Start = time.time()
            for term in query:
                process=Popen(["grep",'-lr','-w',term,'alldocs'],stdout=PIPE)
                s=str(process.stdout.read())
                list_for_a_term=list(set(s.split("\n")))
                list_for_a_term.remove('')
                list_for_a_term=[doc.replace('alldocs/','') for doc in list_for_a_term]
                doc_list.append(list_for_a_term)
            doc_list=list(reduce(set.intersection, [set(item) for item in doc_list ]))
            End=time.time()
            given_doc_list=given_output[query_index]
            total_retrieved=len(doc_list)
            num_true_positive=len(set(given_doc_list)&set(doc_list))
            if(total_retrieved>0):
                precision=(num_true_positive/total_retrieved)*100
                print("Precision: "+str(precision)+"  "+query_index)
                precisions.append(precision)
            recall=((float)(num_true_positive/50.00))*100.00
            recalls.append((query_index,recall))
            time_file.write(query_index+" "+str(End-Start)+"\n")
            for doc in doc_list:
                output_file.write(query_index+" "+doc)
                output_file.write("\n")
average_precision=sum(precisions)/len(precisions)
print("Average precision is: "+str(average_precision))
print(recalls)
fp.close()
output_file.close()
time_file.close()

    

    

['701', 'describe', 'history', 'oil', 'industry']
Precision: 4.1095890411  701
['702', 'pearl', 'farming', 'operations', 'actual', 'farming', 'operations', 'described', 'culturing', 'pearls', 'japanese', 'pearl', 'productions', 'status', 'pearl', 'farming', 'production']
['704', 'goals', 'political', 'views', 'green', 'party']
Precision: 18.1818181818  704
['705', 'identify', 'efforts', 'proposed', 'undertaken', 'world', 'governments', 'seek', 'reduction', 'iraqs', 'foreign', 'debt']


KeyboardInterrupt: 